In [1]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 1319270831416312176
 xla_global_id: -1, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14415560704
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 17042471542350177192
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
 xla_global_id: 416903419]

In [2]:
!nvcc --version
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-po4khzpu
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-po4khzpu
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4304 sha256=c407c05f0110aeb4fe86ea37be1e9039c73c8cfc0b0c0992283b4d72885e8ae3
  Stored in directory: /tmp/pip-ephem-wheel-cache-od_byhsh/wheels/f3/08/cc/e2b5b0e1c92df07dbb50a6f024a68ce090f5e7b2316b41756d
Successfully built NVCCPlugin
cr

In [3]:
 ! df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay          79G   23G   56G  30% /
tmpfs            64M     0   64M   0% /dev
shm             5.7G     0  5.7G   0% /dev/shm
/dev/root       2.0G  1.1G  841M  58% /sbin/docker-init
tmpfs           6.4G   40K  6.4G   1% /var/colab
/dev/sda1        80G   49G   32G  61% /opt/bin/.nvidia
tmpfs           6.4G     0  6.4G   0% /proc/acpi
tmpfs           6.4G     0  6.4G   0% /proc/scsi
tmpfs           6.4G     0  6.4G   0% /sys/firmware


In [4]:
 !cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2199.998
cache size	: 56320 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa mmio_stale_data retbleed
bogomips	: 4399.99
clflush size	: 64
cache_alignment	: 64
address sizes

# Kernel Code

In [ ]:
%%cu

#include <cstdio>
#include <iostream>
#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include "cuda_runtime_api.h"
#include <stdio.h>
#include <cuda.h>
#include <stdlib.h>
#include <math.h>
#include <time.h>

using namespace std;

#define BLOCK_DIM 16

void initMat(float *matrix, int size) {
    for (int i = 0; i < size*size; i++) {
        matrix[i] = 0;
    }
}

void randMat(float *matrix, int size) {
    srand((unsigned int)time(NULL));
    for (int i = 0; i < size; ++i) {
        for (int j = 0; j < size; ++j) {
            matrix[i * size + j] = ((float)rand() / (float)(RAND_MAX)) * 100;
        }
    }    
}

void printMat(float *matrix, int size) {
    for (int i = 0; i < size; ++i) {
        for (int j = 0; j < size; ++j)
            printf("%.2f  ", matrix[i * size + j]);
        printf("\n");
    }    
}


__global__ void luFactorizationDev(float* matrix, float* upper, float* lower, int size) {

	for (size_t k = 0; k < BLOCK_DIM; ++k)
	{
		unsigned int x = threadIdx.x;
		unsigned int y = threadIdx.y;
		
		if(x >= k && x < size && y == k)
		{	
			int sum = 0.;
			for (size_t p = 0; p < k; ++p)
			{
				sum += lower[p + k * size] * upper[x + p * size];
			}
			upper[x + k * size] = matrix[x + k * size] - sum;	
		}
		__syncthreads();

		if(y >= k && y < size && x == k)
		{
			int sum = 0.;
			for (size_t p = 0; p < k; ++p)
			{
				sum += lower[p + y * size] * upper[k + p * size];
			}
			lower[k + y * size] = (matrix[k + y * size] - sum) / upper[k + k * size];
		}
		__syncthreads();
	}
}

void luFactorizationHost(float *matrix, float *upper, float *lower, int size) {
	for (int i = 0; i < size; i++) {
		// Upper triangular
		for (int k = i; k < size; k++) {
			int sum = 0;
			for (int j = 0; j < i; j++)
				sum += (lower[i * size + j] * upper[j * size + k]);
			upper[i * size + k] = matrix[i * size + k] - sum;
		}

		// Lower triangular
		for (int k = i; k < size; k++) {
			if (i == k)
				lower[i * size + i] = 1;
			else {
				int sum = 0;
				for (int j = 0; j < i; j++)
					sum += (lower[k * size + j] * upper[j * size + i]);
				lower[k * size + i] = (matrix[k * size + i] - sum) / upper[i * size + i];
			}
		}
	}
}


int main()
{
    float* dev_A, *dev_up, *dev_low;
    float* A, *up, *low;
    int N = 1023;
    int size = N * N * sizeof(float);
    dim3 threadsPerBlock(BLOCK_DIM, BLOCK_DIM);
    dim3 numBlocks(ceil((float)N / threadsPerBlock.x), ceil((float)N / threadsPerBlock.y));
   
    cudaMallocHost((void**)&A, size);
    cudaMallocHost((void**)&up, size);
    cudaMallocHost((void**)&low, size);
         
    cudaMalloc((void**)&dev_A, size);
    cudaMalloc((void**)&dev_up, size);
    cudaMalloc((void**)&dev_low, size);

    randMat(A, N);
    initMat(up, N);
    initMat(low, N);
 
    //printf("Matrix A\n");

    cudaMemcpy(dev_A, A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(dev_up, up, size, cudaMemcpyHostToDevice);
    cudaMemcpy(dev_low, low, size, cudaMemcpyHostToDevice);
 
    cudaEvent_t start, stop;
    float add_et = 0.0;
    float add_bw = 0.0;
    float msec = 0.0;
    for (int i = 0; i < 10; i++) {
      cudaEventCreate(&start);
      cudaEventCreate(&stop);
      cudaEventRecord(start);
      
      luFactorizationDev << <numBlocks, threadsPerBlock >> > (dev_A, dev_up, dev_low, N);
      
      cudaEventRecord(stop);
      cudaEventSynchronize(stop);
      cudaEventElapsedTime(&msec, start, stop);
      add_et += msec;
      add_bw += N*N * 4 * 3 / msec/ 1e6;
      cudaEventDestroy(start);
      cudaEventDestroy(stop);
    }
 
    clock_t begin = clock();
    cudaMemcpy(up, dev_up, sizeof(float)*N*N, cudaMemcpyDeviceToHost);
    cudaMemcpy(low, dev_low, sizeof(float)*N*N, cudaMemcpyDeviceToHost);
    clock_t end = clock();
    add_et += (float)(end - begin) / CLOCKS_PER_SEC;
 
    add_et = add_et / 10;
    add_bw = add_bw / 10;
    printf("Kernel Average Elapsed Time: %.6fs\n", add_et/1000);
    printf("Kernel Average Effective Bandwidth: %fgb/s\n", add_bw);

 
    printf("\nKernel\n");
    printf("Lower matrix\n");
    //printMat(low, N);
 
    printf("\n");
    printf("Upper matrix\n");
    //printMat(up, N);
 
    
    add_et = 0;
    for (int i = 0; i < 10; i++) {
      clock_t begin = clock();
      luFactorizationHost(A, up, low, N);
      clock_t end = clock();
      add_et += (float)(end - begin) / CLOCKS_PER_SEC;
    }
    
    add_et = add_et / 10;
 
    printf("Host Average Elapsed Time: %.6fs\n", add_et); 
 
    
    printf("\nHost\n");
    printf("Lower matrix\n");
    //printMat(low, N);

    printf("\n");
    printf("Upper matrix\n");
    //printMat(up, N);
 
    return 0;

}